<a href="https://colab.research.google.com/github/otanet/NLP_seminar_20201022/blob/main/%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E6%99%82%E4%BB%A3%E3%81%AE%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E5%89%8D%E5%87%A6%E7%90%86_(1)_Sentencepiece_%E3%82%B3%E3%83%9E%E3%83%B3%E3%83%89%E3%83%A9%E3%82%A4%E3%83%B3%E3%83%84%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 深層学習時代のテキスト前処理:(1) Sentencepiece コマンドラインツール 

Sentencepiece （C++）版をビルド、インストールし、コマンドラインツールとしてサブワードの学習、トークン化(エンコード), 脱トークン化(デコード) を行います。



## インストール

sentencepiece のソースをgithub からダウンロードし、ビルド・インストールを行います。debian のパッケージ化が進んでいるため、近い将来 apt-get 経由でインストールできるかもしれません。

In [ ]:
!git clone https://github.com/google/sentencepiece.git
%cd sentencepiece
!mkdir build
%cd build
!cmake ..
!make -j20
!make install
!ldconfig
!spm_train --help

Cloning into 'sentencepiece'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 3333 (delta 31), reused 36 (delta 16), pack-reused 3273
Receiving objects: 100% (3333/3333), 27.06 MiB | 22.96 MiB/s, done.
Resolving deltas: 100% (2308/2308), done.
/content/sentencepiece
/content/sentencepiece/build
-- VERSION: 0.1.93
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile featu

## サブワードモデルの学習

「吾輩は猫である」のサンプル文書を(sentencepiece/data/wagahaiwa_nekodearu.txt)用いて、サブワードモデルを学習します。

学習には spm_train コマンドを用います。最低限以下のオプションを与えます


*   --vocab_size=8000:  サブワードの語彙サイズ
*   --input=sentencepiece/data/wagahaiwa_nekodearu.txt: 学習データ(一文一行)
*   --model_prefix=m: 出力モデルファイル (m.model/m.vocabファイルが作成されます。)




In [ ]:
%cd ..
!spm_train --vocab_size=8000 --input=data/wagahaiwa_nekodearu.txt --model_prefix=m

/content/sentencepiece
sentencepiece_trainer.cc(79) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/wagahaiwa_nekodearu.txt
  input_format: 
  model_prefix: m
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1
  escape_whi

## トークン化 (エンコード)

学習されたモデルを用いて、任意の文をエンコード(単語分割)します。分割には spm_encode コマンドを用います。--model オプションにてモデルを指定します。分割対象の文は、標準入力もしくはファイルとして与えます。


In [ ]:
!echo "吾輩は猫である。名前はまだない。" | spm_encode --model=m.model
!head -20 data/wagahaiwa_nekodearu.txt |  spm_encode --model=m.model

▁吾輩は 猫 である 。 名前 はまだ ない 。
▁吾輩は 猫 である
▁ 夏 目 漱 石
▁ ------------- -------------- -------------- --------------
▁ 【 テ キ ス ト 中に 現 れる 記 号 について 】
▁ 《 》 : ル ビ
▁( 例 ) 吾輩 《 わが はい 》 は猫である
▁ | : ルビの 付 く 文字 列 の 始ま りを 特 定 する 記 号
▁( 例 ) 一番 | 獰 悪 《 どう あ く 》 な 種 族 であった
▁[# ] : 入 力 者 注 ▁ 主 に 外 字 の 説明 や 、 傍点 の位置 の 指 定
▁( 数 字 は 、 J I S ▁ X ▁ 0 2 1 3 の 面 区 点 番 号 または U ni co d e 、 底本 の ページ と 行 数 )
▁( 例 ) ※[#「 言 + 墟 のつくり 」 、 第 4 水準 2 - 8 8 - 7 4 ]
▁ 〔 〕 : アクセント分解 された 欧 文 を か こ む
▁( 例 ) 〔 Q u id ▁a li u d ▁ es t ▁ m u li e r ▁ ni s i ▁a mic iti ae & ▁ i ni mic a 〕
▁ アクセント分解 について の 詳 細 は下 記 U R L を 参 照 して くださ い
▁ ht t p : / / w ww . a o z o r a . g r . j p / a c c e n t _ s e p a r a t i o n . ht m l
▁ ------------- -------------- -------------- --------------
▁[# 8 字下げ ] 一 [#「 一 」 は中 見出し ]
▁ 吾輩 《 わが はい 》 は猫である 。 名前 はまだ 無い 。
▁ どこ で 生れ たか とん と 見当 《 けん とう 》 が つか ぬ 。 何でも 薄暗 い じめ じめ した 所 で ニャー ニャー 泣 いて いた 事 だけは 記憶している 。 吾輩は ここで 始めて 人間 というもの を見た 。 しかも あとで 聞くとそれは 書生 という 人間 中 で 一番 | 獰 悪 《 どう あ く 》 な 種 族 であった そうだ 。 この 

--output_format=id とすることで、トークンに対応するIDを直接出力することが可能です。

In [ ]:
!echo "吾輩は猫である。名前はまだない。" | spm_encode --model=m.model --output_format=id
!head -20 data/wagahaiwa_nekodearu.txt |  spm_encode --model=m.model --output_format=id

1130 107 32 5 720 2230 52 5
1130 107 32
36 3948 135 7300 652
36 7085 3426 3426 3426
36 7800 2129 7717 444 1719 611 2514 554 3914 4643 966 7716
36 4 3 1731 624 3352
3066 940 836 301 4 2020 145 3 6015
36 44 1731 5225 844 26 783 2089 7 4148 2540 3905 1191 34 3914 4643
3066 940 836 632 44 7191 559 4 251 45 26 3 20 1285 7964 818
175 37 1731 331 281 237 3282 36 874 9 526 651 7 1422 41 6 66 6480 7 1013 1191
3066 543 651 13 6 7710 7709 7713 36 7091 36 4621 227 719 718 7 285 5305 625 1350 4643 2018 7134 3434 4026 5505 2617 6 2663 7 3305 14 211 543 836
3066 940 836 895 236 1098 7805 2659 8 6 580 1297 896 227 7991 782 782 7991 2063 1297 37
36 4666 4670 1731 7081 930 7149 415 12 16 58 84
3066 940 836 4666 7090 2352 5599 4959 4680 2352 5505 36 5679 2631 36 3984 2352 4680 2617 1927 36 3434 2349 2983 4959 3987 4671 4752 5547 36 2983 3434 3987 1266 4670
36 7081 966 7 5542 2066 1255 3914 7134 7712 7802 12 1009 5207 33 3709 17
36 5553 2631 3421 1731 2350 2350 6984 6986 1925 1266 1923 6964 1923 1927 1266

## 脱トークン化 (デコード)
分割済みのトークン列やID列から元の文をdecode(復元)します。decode には spm_decode コマンドを用います。

In [ ]:
!echo "▁吾輩は 猫 である 。 名前 はまだ ない 。" | spm_decode --model=m.model
!echo "1130 107 32 5 720 2230 52 5" | spm_decode --model=m.model --input_format=id

吾輩は猫である。名前はまだない。
吾輩は猫である。名前はまだない。
